In [7]:
from dbconn_base2 import querydb, querydbtopandas 
import pandas as pd 

In [8]:
#con esto veo el mes 11
menor = "2019-07-01"
mayor = "2019-12-30"

In [46]:
df = querydbtopandas(f"""SELECT DISTINCT
    pln.deno_prov AS provincia,
    cpe.prestad prestador,
    pre.ape_razon AS prestador_nombre, 
    afilmed.dbo.afn_ape_nom_afilmed_de_presmed (cpe.contra, cpe.inte , null, 'S') afiliado,
    afilmed.dbo.afn_contra_afilmed_de_presmed(cpe.contra) + afilmed.dbo.afn_inte_de_presmed(cpe.inte) contra,
    cpe.ambu_inter,
    cpe.prestac ,
    prestac.nombre prestacion_nombre,
    prestac.sinonimo prestacion_sinonimo,
    prestac.complejidad complejidad,
    cpe.cantidad ,
    con.deno AS concepto,
    valor_referencia, 
    valor_presupuestado, 
    valor_excepcion, 
    valor_liqui,
    mon.descri AS moneda ,
    cpe.fecha_vig,
    CONVERT(CHAR(10), cpe.fecha_vig, 3) ||' - '||CONVERT(CHAR(10), cpe.fecha_fin_vig, 3) AS vigencia_desde_hasta,
    cpe.obser , 
    cpe.conve_excep    
    
      FROM conve_prestaciones_exc cpe
      INNER JOIN prestad_lugares pl ON cpe.prestad = pl.prestad 
       AND cpe.lugar = pl.lugar 
       AND cpe.baja_fecha is null
      INNER JOIN prestad_lugares_norm pln ON cpe.prestad = pln.prestad 
       AND cpe.lugar = pln.lugar  
      INNER JOIN zgeo_zona_localidad zzl ON zzl.loca = pln.cod_loca
      INNER JOIN prestadores pre ON cpe.prestad = pre.prestad
      LEFT  JOIN conve_valores_exc cve ON cpe.conve_excep = cve.conve_excep
       AND cve.baja_fecha is null 
      LEFT JOIN conceptos con ON cve.concep = con.concep
      LEFT JOIN moneda mon ON cve.moneda = mon.moneda
      LEFT JOIN motivos_exc mot ON mot.id_motivo = cve.id_motivo 
      
      LEFT JOIN v_nom_prestacion_version_vig prestac ON cpe.prestac = prestac.codigoAsignado
      
      WHERE (cpe.fecha_vig >= ("{menor}") AND cpe.fecha_vig <= ("{mayor}"))""")

In [47]:
len(df)  #2161 

2161

In [24]:
q_excepciones = df.groupby(["prestador", "prestador_nombre"])["cantidad"].sum()

In [25]:
q_afiliados =  df.groupby(["prestador", "prestador_nombre"])["afiliado"].nunique()

In [26]:
valor_presupuestado =  df.groupby(["prestador", "prestador_nombre"])["valor_presupuestado"].sum()

In [27]:
valor_excepcion =  df.groupby(["prestador", "prestador_nombre"])["valor_excepcion"].sum()

In [28]:
valor_liqui =  df.groupby(["prestador", "prestador_nombre"])["valor_liqui"].sum()

In [29]:
final = pd.concat([q_excepciones,q_afiliados,valor_presupuestado,valor_excepcion,valor_liqui],axis=1).reset_index()

In [30]:
final = final.sort_values(by= "cantidad",ascending= False)

In [31]:
final.reset_index(drop=True, inplace=True)

In [37]:
(pd.read_csv("correcto.csv", index_col=0).round(2) == final.round(2)).all()

prestador              True
prestador_nombre       True
cantidad               True
afiliado               True
valor_presupuestado    True
valor_excepcion        True
valor_liqui            True
dtype: bool